In [1]:
import pandas as pd
from newspaper import Article
import requests
import numpy as np

In [2]:
# Import the appropriate functions from the vader library 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

### SentimentScoreAnalyzer

The `SentimentScoreAnalyzer` class analyzes the sentiment of news articles for a specific stock. Given a set of keywords and a stock symbol, it identifies relevant articles, fetches article text, and calculates sentiment scores using VADER.

#### Key Methods:
- `map_article_to_stock(article_title, article_text)`: Checks if the article contains any keywords related to the specified stock, returning the stock symbol if it does.
- `fetch_article_text(url)`: Retrieves and parses the full text of an article from a URL. Handles common HTTP errors and issues during retrieval.
- `add_sentiment_scores_to_df(df, filename)`: Processes a DataFrame of article URLs and titles, calculates sentiment scores for articles that match the stock's keywords, and saves the results with sentiment scores in a CSV file.

This class is useful for quantifying sentiment in financial news articles to gauge market sentiment on a specific stock.


In [77]:
class SentimentScoreAnalyzer:
    def __init__(self, keywords, stock_symbol):
        self.keywords = keywords
        self.stock_symbol = stock_symbol
        self.analyzer = SentimentIntensityAnalyzer()

    def map_article_to_stock(self, article_title, article_text):
        combined_text = f"{article_title} {article_text}".lower()
        for keyword in self.keywords:
            if keyword.lower() in combined_text:
                return self.stock_symbol
        return None

    def fetch_article_text(self, url):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
        }
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()  # Raises HTTP errors if they occur
            
            article = Article(url)
            article.set_html(response.text)
            article.parse()
            return article.text
        except requests.exceptions.HTTPError as e:
            status_code = e.response.status_code
            if status_code == 403:
                print(f"Access forbidden for URL: {url}")
            elif status_code == 401:
                print(f"Unauthorized access for URL: {url}")
            elif status_code == 404:
                print(f"Article not found at URL: {url}")
            elif status_code == 422:
                print(f"Unprocessable content at URL: {url}")
            elif status_code == 502:
                print(f"Bad gateway error for URL: {url}. Retrying might help.")
            else:
                print(f"HTTP error occurred for URL {url}: {e}")
            return None
        except Exception as e:
            print(f"Error fetching article text for URL {url}: {e}")
            return None

    def add_sentiment_scores_to_df(self, df, filename):
        sentiment_scores = []
        
        for _, row in df.iterrows():
            title = row['title']
            url = row['url']
            article_text = self.fetch_article_text(url)

            content = title + " " + article_text if article_text else title
            if self.map_article_to_stock(title, article_text):
                sentiment = self.analyzer.polarity_scores(content)
                sentiment_scores.append(sentiment['compound'])
            else:
                sentiment_scores.append(0)

        df['compound sentiment score'] = sentiment_scores
        df.dropna(subset=['compound sentiment score'], inplace=True)
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Sentiment scores saved to {filename}")

In [68]:
data_it = pd.read_csv('/Users/rishabhbhardwaj/Desktop/Bootcamp project/NewsAPI/IT_all_news.csv')
df_it = pd.DataFrame(data_it)

# Convert the date_time column to datetime format
df_it['publishedAt'] = pd.to_datetime(df_it['publishedAt'])

# Sort by date_time in ascending order (past to future)
df_it = df_it.sort_values(by='publishedAt').reset_index(drop=True).drop_duplicates()

In [75]:
data_cons = pd.read_csv('/Users/rishabhbhardwaj/Desktop/Bootcamp project/NewsAPI/consumers_all_news.csv')
df_cons = pd.DataFrame(data_cons)

# Convert the date_time column to datetime format
df_cons['publishedAt'] = pd.to_datetime(df_cons['publishedAt'])

# Sort by date_time in ascending order (past to future)
df_cons = df_cons.sort_values(by='publishedAt').reset_index(drop=True).drop_duplicates()

In [80]:
data_comm = pd.read_csv('/Users/rishabhbhardwaj/Desktop/Bootcamp project/NewsAPI/communication_all_news.csv')
df_comm = pd.DataFrame(data_comm)

# Convert the date_time column to datetime format
df_comm['publishedAt'] = pd.to_datetime(df_cons['publishedAt'])

# Sort by date_time in ascending order (past to future)
df_comm = df_comm.sort_values(by='publishedAt').reset_index(drop=True).drop_duplicates()

In [62]:
keywords = ["microsoft","msft","bill gates","gates","information","technology","cybersecurity"
            ," ai ","artificial","windows","intel","software","silicon","chips"]
analyzer = SentimentScoreAnalyzer(keywords, 'MSFT')
result_df = analyzer.add_sentiment_scores_to_df(df_msft, "msft_sentiment_scores.csv")

Error fetching article text: 403 Client Error: Forbidden for url: https://www.business-standard.com/world-news/us-president-biden-discourages-israel-from-attacking-iran-s-oil-fields-124100500089_1.html


In [69]:
keywords = ["apple","aapl","tim cook","cook","information","technology","cybersecurity"
            ," ai ","artificial","macos","macbook","software","iphone","chips"]
analyzer = SentimentScoreAnalyzer(keywords, 'AAPL')
result_df = analyzer.add_sentiment_scores_to_df(df_it, "aapl_sentiment_scores.csv")

Error fetching article text: 403 Client Error: Forbidden for url: https://www.business-standard.com/world-news/us-president-biden-discourages-israel-from-attacking-iran-s-oil-fields-124100500089_1.html


In [70]:
keywords = ["nvidia","nvda","jensen huang","information","technology","supercomputing"
            ," ai ","artificial","macos","graphics processing unit (GPU)","software"
            ,"deep learning","autonomous vehicles","gaming and esports","edge computing"]
analyzer = SentimentScoreAnalyzer(keywords, 'NVDA')
result_df = analyzer.add_sentiment_scores_to_df(df_it, "nvda_sentiment_scores.csv")

Error fetching article text: 403 Client Error: Forbidden for url: https://www.business-standard.com/world-news/us-president-biden-discourages-israel-from-attacking-iran-s-oil-fields-124100500089_1.html


In [78]:
keywords = ["amazon","amzn","jeff bezos","bezos","retail","supply chain","household",
            "e-commerce","aws"," ai ","delivery","streaming","digital"]
analyzer = SentimentScoreAnalyzer(keywords, 'AMZN')
result_df = analyzer.add_sentiment_scores_to_df(df_cons, "amzn_sentiment_scores.csv")

Access forbidden for URL: https://undark.org/2024/09/23/h5n1-old-drug-protect-against-new-pandemic/
Unauthorized access for URL: https://consent.yahoo.com/v2/collectConsent?sessionId=1_cc-session_a5bcc80c-318c-43dd-b61d-0f75e161cc92
Unprocessable content at URL: https://phys.org/news/2024-09-cement-coastline-ecosystems.html
Unprocessable content at URL: https://phys.org/news/2024-09-chief-sea-threatens-tide-misery.html
Article not found at URL: https://abcnews.go.com/US/wireStory/hurricane-helene-unusual-fujiwhara-effect-114208917
Unprocessable content at URL: https://phys.org/news/2024-09-regional-climate-essential-effective-policy.html
Bad gateway error for URL: https://springwise.com/innovation/sustainability/modular-floating-land-tackles-climate-change/. Retrying might help.
Access forbidden for URL: https://parade.com/health/top-vaccine-myth-to-stop-believing-according-to-infectious-disease-experts
Sentiment scores saved to amzn_sentiment_scores.csv


In [82]:
keywords = ["alphabet","goog","broadband","5g","media"," ai ", "sundar pichai","pichai","cloud computing"
            ,"data privacy","data","youtube","android os","quantum computing","search engine optimization"]
analyzer = SentimentScoreAnalyzer(keywords, 'GOOG')
result_df = analyzer.add_sentiment_scores_to_df(df_comm, "goog_sentiment_scores.csv")

Bad gateway error for URL: https://springwise.com/innovation/sustainability/modular-floating-land-tackles-climate-change/. Retrying might help.
Unprocessable content at URL: https://phys.org/news/2024-10-climate-escalating-crisis-urges-action.html
Sentiment scores saved to goog_sentiment_scores.csv


### SentimentScoreAnalyzerCSV

The `SentimentScoreAnalyzerCSV` class analyzes sentiment in news article headlines related to a specified stock, using a list of keywords and customized sentiment scores for relevant geopolitical, economic, climate, and healthcare terms. This helps capture a more nuanced sentiment based on context-specific terms.

#### Key Components:
- **Custom Sentiment Lexicon**: In the `__init__` method, a dictionary of relevant terms is created with predefined sentiment scores, reflecting positive or negative connotations associated with events like "pandemic," "recession," "renewable energy," etc. Each term's score is then adjusted with a small random variation to add realism.
- **map_article_to_stock(text)**: Checks if any keywords related to the stock are present in the article's text. Returns the stock symbol if a match is found.
- **add_sentiment_scores_to_df(df, start_date, end_date, filename)**: Processes a DataFrame of headlines, applies sentiment analysis to articles within a specified date range, and calculates an average sentiment score for each row based on the adjusted lexicon. Saves the DataFrame with added sentiment scores to a CSV file.

This class is useful for generating sentiment analysis tailored to stock performance influenced by specific global events and trends.


In [3]:
class SentimentScoreAnalyzerCSV:
    def __init__(self, keywords, stock_symbol):
        self.keywords = keywords
        self.stock_symbol = stock_symbol
        self.analyzer = SentimentIntensityAnalyzer()

        # Define relevant keywords with associated sentiment scores
        news_terms = {
    # Geopolitical News
    "war": -0.9, "conflict": -0.85, "peace": 0.85, "treaty": 0.8,
    "border tension": -0.75, "military intervention": -0.8, "sanctions": -0.75,
    "alliance": 0.7, "nuclear deal": -0.8, "ceasefire": 0.8, "diplomacy": 0.7,
    "foreign aid": 0.6, "refugee crisis": -0.85, "arms trade": -0.7, "geopolitical risk": -0.85,
    "territorial dispute": -0.8, "trade war": -0.8, "embargo": -0.75,

    # Elections
    "election": 0.4, "vote": 0.5, "ballot": 0.5, "presidential race": 0.6,
    "campaign": 0.4, "election fraud": -0.8, "polling": 0.3, "election results": 0.5,
    "candidate": 0.4, "recount": -0.4, "voter turnout": 0.5, "political debate": 0.4,
    "incumbent": 0.3, "nomination": 0.4, "primaries": 0.3, "exit poll": 0.4,
    "voting rights": 0.6, "referendum": 0.4, "democratic process": 0.6,

    # Climate
    "climate change": -0.75, "global warming": -0.8, "carbon emission": -0.7,
    "sustainability": 0.75, "renewable energy": 0.85, "pollution": -0.75,
    "greenhouse gases": -0.7, "environmental protection": 0.8, "deforestation": -0.7,
    "biodiversity": 0.7, "ocean acidification": -0.8, "conservation": 0.75,
    "wildfire": -0.85, "natural disaster": -0.9, "reforestation": 0.8,
    "climate summit": 0.7, "clean energy": 0.85, "carbon footprint": -0.6,
    "environmental regulation": 0.65, "eco-friendly": 0.7, "fossil fuels": -0.65,

    # Pandemic
    "pandemic": -0.95, "virus outbreak": -0.9, "quarantine": -0.85,
    "vaccine": 0.85, "public health": 0.75, "lockdown": -0.8,
    "COVID-19": -0.95, "infection rate": -0.85, "healthcare system": -0.7,
    "herd immunity": 0.75, "social distancing": -0.6, "contact tracing": 0.65,
    "vaccine rollout": 0.8, "booster shot": 0.7, "pandemic relief": 0.75,
    "mask mandate": -0.7, "essential workers": 0.7, "curfew": -0.75,
    "remote work": 0.6, "mental health crisis": -0.8,

    # Government Policies
    "government policy": 0.6, "reform": 0.65, "tax cut": 0.7,
    "stimulus": 0.85, "interest rate hike": -0.7, "inflation": -0.75,
    "economic growth": 0.85, "recession": -0.85, "unemployment": -0.8,
    "job growth": 0.75, "healthcare reform": 0.7, "infrastructure bill": 0.8,
    "foreign policy": 0.65, "trade agreement": 0.7, "import tariffs": -0.65,
    "subsidy": 0.6, "welfare program": 0.7, "minimum wage": 0.7,
    "labor law": 0.6, "regulatory policy": 0.55, "fiscal policy": 0.65,
    "budget deficit": -0.7, "monetary policy": 0.65, "public spending": 0.6,
    "debt ceiling": -0.65, "housing policy": 0.7
}


        # Generate random scores with 5% standard deviation around each base score
        randomized_news_terms = {
            term: round(np.random.normal(loc=score, scale=abs(score) * 0.05), 3) 
            for term, score in news_terms.items()
        }
        self.analyzer.lexicon.update(randomized_news_terms)

    def calculate_weighted_average_sentiment(self, scores):
        if not scores:
            return None
        sum_score = sum(abs(score) for score in scores)
        return sum(score * abs(score) for score in scores) / sum_score if sum_score != 0 else 0

    def map_article_to_stock(self, text):
        text = text.lower()
        return self.stock_symbol if any(keyword.lower() in text for keyword in self.keywords) else None

    def add_sentiment_scores_to_df(self, df, start_date, end_date, filename):
        # Convert 'Date' column to datetime and filter the date range
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        mask = (df['date'] >= pd.to_datetime(start_date)) & (df['date'] <= pd.to_datetime(end_date))
        df = df[mask].copy()

        # Dictionary to hold daily sentiment scores for each type
        daily_sentiment_scores = {}

        # Process each row and calculate sentiment for each headline
        for _, row in df.iterrows():
            date = row['date']
            headline = row['headline']
            
            if pd.isna(headline) or not isinstance(headline, str):
                continue
            
            if self.map_article_to_stock(headline):
                # Calculate sentiment scores for the headline
                sentiment = self.analyzer.polarity_scores(headline)
                pos_score = sentiment['pos']
                neg_score = sentiment['neg']
                neu_score = sentiment['neu']
                compound_score = sentiment['compound']

                # Initialize dictionary entry if date is not yet present
                if date not in daily_sentiment_scores:
                    daily_sentiment_scores[date] = {'pos': [], 'neg': [], 'neu': [], 'compound': []}

                # Append scores to respective lists
                daily_sentiment_scores[date]['pos'].append(pos_score)
                daily_sentiment_scores[date]['neg'].append(neg_score)
                daily_sentiment_scores[date]['neu'].append(neu_score)
                daily_sentiment_scores[date]['compound'].append(compound_score)

        # Compute weighted average sentiment for each score type per date
        results = []
        for date, scores in daily_sentiment_scores.items():
            weighted_avg_pos = self.calculate_weighted_average_sentiment(scores['pos'])
            weighted_avg_neg = self.calculate_weighted_average_sentiment(scores['neg'])
            weighted_avg_neu = self.calculate_weighted_average_sentiment(scores['neu'])
            weighted_avg_compound = self.calculate_weighted_average_sentiment(scores['compound'])
            
            # Only append the result if the weighted compound score is non-zero
           
            results.append({
                    'Date': date,
                    'Ticker': self.stock_symbol,
                    'weighted positive sentiment score': weighted_avg_pos,
                    'weighted negative sentiment score': weighted_avg_neg,
                    'weighted neutral sentiment score': weighted_avg_neu,
                    'weighted compound sentiment score': weighted_avg_compound
                })

        # Convert results to DataFrame and save to CSV
        sentiment_df = pd.DataFrame(results)
        sentiment_df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Sentiment scores saved to {filename}")

In [4]:
buzzwords_dict = {
    "GOOG": [
        # Google-specific
        "Google", "Search Engine", "Google Maps", "Gmail", "YouTube", "Android", "Chrome", "Google Drive", "Alphabet Inc.",
        "Google Cloud", "Google Assistant", "Google Ads", "Google Analytics", "Pixel phone", "Google Home", 
        "DeepMind", "Waymo", "Nest", "DoubleClick", "Google Now", "Google Glass", "YouTube Red", "Google Earth",
        "AdSense", "Google Play", "AI research", "Machine Learning", "TensorFlow", "Google Photos", "Google Ventures",

        # Industry & Market-related
        "search engine optimization", "online advertising", "cloud services", "data privacy", "personalization", 
        "self-driving cars", "AI and ML", "deep learning", "technology investments", "digital marketing", 
        "streaming video", "smart home", "cloud computing", "digital transformation", "market expansion", 
        "user engagement", "mobile ecosystem", "R&D investment", "technology disruption", "quarterly earnings",
        
        # Broader technology & trends
        "AI revolution", "tech giant", "digital media", "advertising revenue", "online content", "search trends", 
        "data-driven decisions", "smartphones", "app ecosystem", "innovation", "consumer tech", "VR", "IoT devices",
        "big data", "privacy policy", "voice search", "computing power", "data security", "scalability",

        # Geopolitical news
        "sanctions", "international trade", "export restrictions", "foreign policy", "tariffs", "market regulation",
        
        # Elections
        "election misinformation", "political ads", "campaign outreach", "social media influence", "voter engagement",

        # Climate
        "carbon footprint", "sustainable computing", "green technology", "energy efficiency", "data center emissions",

        # Pandemic
        "remote work tools", "digital meetings", "contact tracing apps", "healthcare data", "public health innovation",

        # Government policies
        "antitrust regulation", "data protection laws", "tech regulation", "net neutrality", "content moderation policies"
    ],
    "MSFT": [
        # Microsoft-specific
        "Microsoft", "Windows OS", "Microsoft Office", "Azure", "Surface", "Xbox", "Bing", "LinkedIn acquisition",
        "Cortana", "OneDrive", "Visual Studio", "SharePoint", "Outlook", "Teams", "Internet Explorer", "MSN", "HoloLens",
        "Skype", "Dynamics", "Microsoft Edge", "SQL Server", "Power BI", "Active Directory", "Silverlight", "DirectX",
        
        # Industry & Market-related
        "cloud computing", "SaaS", "cloud services", "software licensing", "operating system", "business productivity",
        "gaming market", "data analytics", "cybersecurity", "enterprise solutions", "technology adoption",
        "data center", "AI and machine learning", "collaboration tools", "digital transformation", "partnership",
        "acquisition", "quarterly earnings", "market share", "R&D investment", "stock growth", "enterprise software",
        
        # Broader technology & trends
        "technology innovation", "tech giant", "productivity tools", "smartphones", "cross-platform", "software developer",
        "digital transformation", "personal computing", "enterprise adoption", "user experience", "mobile apps",
        "cloud storage", "cyber resilience", "digital workplace", "computing power", "future of work",

        # Geopolitical news
        "trade sanctions", "global supply chain", "foreign investment", "tech imports", "international policy",

        # Elections
        "voter data privacy", "political targeting", "fact-checking tools", "campaign software", "election security",

        # Climate
        "carbon neutrality", "renewable energy adoption", "sustainable data centers", "green cloud computing", "climate pledges",

        # Pandemic
        "remote work software", "virtual collaboration", "pandemic resilience", "digital healthcare solutions",

        # Government policies
        "antitrust scrutiny", "privacy legislation", "tech policy changes", "cybersecurity regulations", "intellectual property"
    ],
    "AMZN": [
        # Amazon-specific
        "Amazon", "e-commerce", "AWS", "Prime", "Kindle", "Amazon Echo", "Alexa", "Amazon Fresh", "Amazon Web Services",
        "Fire TV", "Prime Video", "Whole Foods", "Fulfillment Centers", "Prime Day", "marketplace", "third-party sellers",
        "logistics", "supply chain", "Amazon Go", "Amazon Basics", "subscription service", "Prime Membership",
        "Amazon Studios", "self-publishing", "Kindle Unlimited", "Amazon Music", "AWS Lambda", "Redshift", "S3 Storage",
        
        # Industry & Market-related
        "cloud computing", "digital commerce", "online retail", "last-mile delivery", "subscription economy",
        "streaming services", "logistics innovation", "digital transformation", "consumer behavior", "data analytics",
        "retail innovation", "cloud services", "media streaming", "warehousing", "supply chain management",
        "technology disruption", "AI and machine learning", "smart home", "customer experience", "fulfillment services",
        
        # Broader technology & trends
        "e-commerce growth", "consumer electronics", "tech giant", "retail sales", "online shopping", 
        "home automation", "digital assistants", "voice-activated", "data privacy", "digital marketing", 
        "smart devices", "IoT", "big data", "consumer insights", "logistics automation", "digital subscriptions",
        "cyber Monday", "Black Friday", "technology innovation", "R&D investment",

        # Geopolitical news
        "trade tariffs", "cross-border sales", "international markets", "customs regulations", "foreign policy",

        # Elections
        "consumer rights", "labor regulations", "union support", "tax policies", "political lobbying",

        # Climate
        "carbon footprint", "eco-friendly packaging", "green logistics", "renewable energy", "sustainability initiatives",

        # Pandemic
        "e-commerce boom", "contactless delivery", "essential goods", "remote work", "supply chain disruption",

        # Government policies
        "antitrust investigations", "tax laws", "labor rights", "data privacy regulations", "environmental policies"
    ],
    "NVDA": [
        # NVIDIA-specific
        "NVIDIA", "GPU", "GeForce", "Quadro", "Tesla GPUs", "Tegra", "CUDA", "NVIDIA Shield", "SLI technology", 
        "GameWorks", "Deep Learning", "NVIDIA Drive", "G-Sync", "AI research", "Tensor Cores", "Pascal architecture", 
        "Maxwell architecture", "Kepler architecture", "Graphics card", "NVIDIA Jetson", "NVLink", "DLSS technology",
        "NVIDIA GRID", "PhysX", "CUDA cores", "3D Vision", "Omniverse", "Founders Edition",

        # Industry & Market-related
        "semiconductors", "AI chips", "data center", "gaming industry", "machine learning", "VR", "virtual reality",
        "augmented reality", "cloud computing", "supercomputing", "autonomous vehicles", "AI hardware", "stock performance",
        "AI market", "deep learning acceleration", "technology innovation", "partnership", "acquisition", "R&D investment",
        "market expansion", "product launch", "shareholder", "stock growth", "revenue growth", "quarterly earnings",
        "competition", "cloud services", "infrastructure", "tech giant", "software stack", "energy efficiency", 
        "compute power", "processing power", "machine vision", "graphical processing", "gaming performance",

        # Broader technology & trends
        "innovation", "AI revolution", "tech innovation", "big data", "machine intelligence", "inference", 
        "performance boost", "edge computing", "IoT devices", "next-gen technology", "consumer electronics", 
        "visual computing", "developer ecosystem", "parallel processing", "scalability", "hardware upgrade",

        # Geopolitical news
        "semiconductor tariffs", "supply chain", "tech export restrictions", "global demand", "foreign markets",

        # Elections
        "technology regulation", "innovation funding", "national security", "foreign investment restrictions",

        # Climate
        "energy efficiency", "green technology", "carbon neutrality", "sustainable production", "renewable materials",

        # Pandemic
        "remote work equipment", "digital learning", "medical imaging", "healthcare AI", "supply chain resilience",

        # Government policies
        "technology export policy", "AI research funding", "IP protection", "data privacy laws", "environmental policy"
    ],
    "AAPL": [
        "iPhone", "iPad", "MacBook", "iPod", "Apple Watch", "AirPods", "iMac", "Mac Pro", "Mac Mini", "Apple TV",
        "App Store", "Apple Music", "iTunes", "iCloud", "Siri", "Touch ID", "Face ID", "Apple Pay", "Apple Pencil",
        "Retina Display", "iOS", "macOS", "OS X", "iLife", "iWork", "Safari", "Lightning Connector",
        "A-series chips", "M-series chips", "HealthKit", "HomeKit", "CarPlay", "iMessage", "AppleCare", 
        "Final Cut Pro", "Logic Pro", "Pro Display XDR", "iBooks", "iPhoto", "GarageBand", "iMovie", 
        "Time Capsule", "AirPort", "Apple Store", "Apple Campus", "Apple Silicon", "Digital Hub",
        
        # Common Apple-related buzzwords
        "Apple event", "product launch", "Apple stock", "Apple earnings", "Steve Jobs", "Tim Cook",
        "product innovation", "brand loyalty", "premium devices", "ecosystem", "user experience",
        "tech giant", "market leader", "tech innovation", "stock performance", "R&D investment",
        "software update", "security features", "quarterly results", "brand reputation", "environmental responsibility",
        "corporate responsibility", "customer privacy", "data encryption", "consumer electronics", "wearable tech",
        "smartphone market", "tablet market", "tech stock", "cloud services", "digital marketplace", "app development",
        "global expansion", "supply chain", "retail strategy", "customer experience", "device upgrade",

        # Geopolitical news
        "international supply chain", "China relations", "global tariffs", "trade disputes", "overseas markets",

        # Elections
        "regulation reform", "antitrust legislation", "data privacy laws", "government hearings", "political lobbying",

        # Climate
        "carbon neutral", "recycled materials", "eco-friendly design", "energy-efficient devices", "green packaging",

        # Pandemic
        "remote work tools", "telehealth", "virtual education", "health monitoring", "supply chain disruptions",

        # Government policies
        "antitrust scrutiny", "data privacy", "right to repair", "technology regulations", "consumer protection"
    ]
}


In [5]:
# Load the CSV file with a specified encoding
df = pd.read_csv('/Users/rishabhbhardwaj/Desktop/Bootcamp project/news data/news_headlines_2000-2024.csv', encoding='ISO-8859-1')
# Specify the date range
start_date = "2000-01-03"
end_date = "2024-11-09"
for stock_symbol, keywords in buzzwords_dict.items():
    analyzer = SentimentScoreAnalyzerCSV(keywords, stock_symbol)

    # Run the analysis and save results
    analyzer.add_sentiment_scores_to_df(df, start_date, end_date, f'sentiment_analysis_results_{stock_symbol}.csv')

Sentiment scores saved to sentiment_analysis_results_GOOG.csv
Sentiment scores saved to sentiment_analysis_results_MSFT.csv
Sentiment scores saved to sentiment_analysis_results_AMZN.csv
Sentiment scores saved to sentiment_analysis_results_NVDA.csv
Sentiment scores saved to sentiment_analysis_results_AAPL.csv


### SentimentScoreAnalyzerStockNews

The `SentimentScoreAnalyzerStockNews` class analyzes sentiment scores for stock-related news headlines, tailored to a specific stock symbol. Using a customized sentiment lexicon of finance-related terms, the class adjusts the lexicon with slight random variations to make sentiment scores more realistic.

#### Key Components:
- **Customized Finance Terms**: Defines important finance terms with strong positive or negative sentiment scores (e.g., "EPS beat," "bankruptcy," "revenue miss"). Each term's score is slightly randomized to add variability.
- **calculate_weighted_average_sentiment(scores)**: Calculates a weighted average sentiment score based on the absolute magnitude of each sentiment score.
- **add_sentiment_scores_to_df(df, start_date, end_date, filename)**: Filters the data by date range and stock symbol, calculates a sentiment score for each relevant headline, and aggregates daily weighted average scores. Results are saved to a CSV file.

This class is useful for measuring sentiment impact on specific stocks based on financial news headlines.


In [7]:
class SentimentScoreAnalyzerStockNews:
    def __init__(self, stock_symbol):
        self.stock_symbol = stock_symbol
        self.analyzer = SentimentIntensityAnalyzer()
        # Define the base sentiment scores
        top_finance_terms = {
            "EPS beat": 0.95,
            "revenue miss": -0.92,
            "downgrade": -0.93,
            "FDA approval": 0.96,
            "positive earnings": 0.96,
            "negative earnings": -0.96,
            "profit warning": -0.92,
            "unexpected loss": -0.97,
            "stock surge": 0.95,
            "profit": 0.95,
            "loss": -0.95,
            "revenue": 0.9,
            "growth": 0.8,
            "decline": -0.8,
            "bullish": 0.9,
            "bearish": -0.9,
            "uptrend": 0.7,
            "downtrend": -0.7,
            "debt": -0.85,
            "earnings": 0.85,
            "bankruptcy": -1.0,
            "expenses": -0.75,
            "investment": 0.75,
            "dividend": 0.9,
            "recession": -0.95,
            "inflation": -0.85,
            "recovery": 0.8,
            "unemployment": -0.9,
            "market share": 0.7
        }
        # Generate random scores with 5% standard deviation around each base score
        randomized_finance_terms = {
            term: round(np.random.normal(loc=score, scale=abs(score) * 0.10), 3) for term, score in top_finance_terms.items()
        }

        self.analyzer.lexicon.update(randomized_finance_terms)

    def calculate_weighted_average_sentiment(self, scores):
        # Compute weighted average based on sentiment magnitude
        if not scores:
            return None
        sum_score = sum(abs(score) for score in scores)
        if sum_score != 0:
            return sum(score * abs(score) for score in scores) / sum_score
        return 0

    def add_sentiment_scores_to_df(self, df, start_date, end_date, filename):
        # Convert 'date' column to datetime and filter the date range and stock symbol
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        mask = (
            (df['Date'] >= pd.to_datetime(start_date)) & 
            (df['Date'] <= pd.to_datetime(end_date)) & 
            (df['Ticker'] == self.stock_symbol)
        )
        df = df[mask].copy()

        # Dictionary to hold daily sentiment scores for each type
        daily_sentiment_scores = {}

        # Process each row and calculate sentiment for each headline
        for _, row in df.iterrows():
            date = row['Date']
            headline = row['Headline']  # Assuming the headline is in the 'headline' column
            
            # Skip if headline is NaN or not a string
            if pd.isna(headline) or not isinstance(headline, str):
                continue

            # Calculate sentiment scores for the headline
            sentiment = self.analyzer.polarity_scores(headline)
            pos_score = sentiment['pos']
            neg_score = sentiment['neg']
            neu_score = sentiment['neu']
            compound_score = sentiment['compound']

            # Initialize dictionary entry if date is not yet present
            if date not in daily_sentiment_scores:
                daily_sentiment_scores[date] = {'pos': [], 'neg': [], 'neu': [], 'compound': []}

            # Append scores to respective lists
            daily_sentiment_scores[date]['pos'].append(pos_score)
            daily_sentiment_scores[date]['neg'].append(neg_score)
            daily_sentiment_scores[date]['neu'].append(neu_score)
            daily_sentiment_scores[date]['compound'].append(compound_score)

        # Compute weighted average sentiment for each score type per date
        results = []
        for date, scores in daily_sentiment_scores.items():
            weighted_avg_pos = self.calculate_weighted_average_sentiment(scores['pos'])
            weighted_avg_neg = self.calculate_weighted_average_sentiment(scores['neg'])
            weighted_avg_neu = self.calculate_weighted_average_sentiment(scores['neu'])
            weighted_avg_compound = self.calculate_weighted_average_sentiment(scores['compound'])
            
            # Include zero sentiment scores
            results.append({
                'Date': date,
                'Ticker': self.stock_symbol,
                'weighted positive sentiment score': weighted_avg_pos,
                'weighted negative sentiment score': weighted_avg_neg,
                'weighted neutral sentiment score': weighted_avg_neu,
                'weighted compound sentiment score': weighted_avg_compound
            })

        # Convert results to DataFrame without dropping rows
        sentiment_df = pd.DataFrame(results)

        # Save the DataFrame with all sentiment scores (including zero) to CSV
        sentiment_df.to_csv(filename, index=False, encoding='utf-8')
        print(f"Sentiment scores saved to {filename}")

In [8]:
# Load the CSV file with a specified encoding
df_new = pd.read_csv('/Users/rishabhbhardwaj/Desktop/Bootcamp project/news data/stock_news_data-2011-2024.csv')
# Specify the date range
start_date = "2011-05-02"
end_date = "2024-11-09"
stock_symbols = ['GOOG','AAPL','MSFT','NVDA','AMZN']
for stock_symbol in stock_symbols:
    analyzer = SentimentScoreAnalyzerStockNews(stock_symbol)

    # Run the analysis and save results
    analyzer.add_sentiment_scores_to_df(df_new, start_date, end_date, f'stock_news_sentiment_analysis_results_{stock_symbol}.csv')

Sentiment scores saved to stock_news_sentiment_analysis_results_GOOG.csv
Sentiment scores saved to stock_news_sentiment_analysis_results_AAPL.csv
Sentiment scores saved to stock_news_sentiment_analysis_results_MSFT.csv
Sentiment scores saved to stock_news_sentiment_analysis_results_NVDA.csv
Sentiment scores saved to stock_news_sentiment_analysis_results_AMZN.csv


In [51]:
df_new

,Ticker,Date,Headline
0,DVA,2011-05-02,Davita (NYSE:DVA): EPS of $0.96 beats by $0.01...
1,CGNX,2011-05-02,Cognex (NASDAQ:CGNX): EPS of $0.32 beats by $0...
2,AEIS,2011-05-02,Advanced Energy Industries (NASDAQ:AEIS):
3,LOCM,2011-05-02,Local.com (NASDAQ:LOCM):
4,CYDEQ,2011-05-02,CyberDefender (CYDE):
...,...,...,...
177678,MRK,2024-03-15,FarmSee Ltd Announces the Completion of a Mino...
177679,NVDA,2024-03-15,"Here's My Top ""Magnificent Seven"" Stock to Buy..."
177680,AAPL,2024-03-15,Tim Cook's comments about iPhone sales in Chin...
177681,AAPL,2024-03-15,Apple to pay $490m to settle claims it misled ...


In [43]:
# Load the CSV file
df = pd.read_csv('/Users/rishabhbhardwaj/Desktop/Bootcamp project/news data/stock_news_data-2011-2024.csv')

# Strip any trailing whitespace from the Date column
df['Date'] = df['Date'].str.strip()

# Convert the Date column to datetime format, coercing any errors to NaT
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

# Verify if there are any NaT values remaining
print(f"Number of NaT entries in Date column after cleaning: {df['Date'].isna().sum()}")

# Optionally, save the cleaned DataFrame back to a CSV file
df.to_csv('/Users/rishabhbhardwaj/Desktop/Bootcamp project/news data/cleaned_stock_news_data-2011-2024.csv', index=False)

Number of NaT entries in Date column after cleaning: 0


In [231]:
df_new

,Ticker,Date,Headline
0,DVA,2011-05-02,Davita (NYSE:DVA): EPS of $0.96 beats by $0.01...
1,CGNX,2011-05-02,Cognex (NASDAQ:CGNX): EPS of $0.32 beats by $0...
2,AEIS,2011-05-02,Advanced Energy Industries (NASDAQ:AEIS):
3,LOCM,2011-05-02,Local.com (NASDAQ:LOCM):
4,CYDEQ,2011-05-02,CyberDefender (CYDE):
...,...,...,...
177678,MRK,2024-03-15,FarmSee Ltd Announces the Completion of a Mino...
177679,NVDA,2024-03-15,"Here's My Top ""Magnificent Seven"" Stock to Buy..."
177680,AAPL,2024-03-15,Tim Cook's comments about iPhone sales in Chin...
177681,AAPL,2024-03-15,Apple to pay $490m to settle claims it misled ...


In [273]:
df_news_recent = pd.read_csv('/Users/rishabhbhardwaj/Desktop/Bootcamp project/stock_news_2024-11_to_2024-03.csv')

In [274]:
df_news_recent

,Unnamed: 0,stock,date,title,text,url
0,0,AMZN,2024-03-16 03:15:00-04:00,Forget Amazon: These 3 Hot Growth Stocks Could...,Toast is harnessing a large market opportunity...,https://www.fool.com/investing/2024/03/16/forg...
1,1,AMZN,2024-03-16 03:22:00-04:00,2 Artificial Intelligence (AI) Stocks to Buy H...,"Amazon is benefiting from AI in two ways, and ...",https://www.fool.com/investing/2024/03/16/2-ai...
2,2,MSFT,2024-03-16 04:00:00-04:00,2 Top Tech Stocks to Buy Hand Over Fist Before...,AI adoption is set to give Microsoft's growth ...,https://www.fool.com/investing/2024/03/16/top-...
3,3,AMZN,2024-03-16 05:50:00-04:00,Eli Lilly Partners With Amazon's Pharmacy Busi...,Eli Lilly recently selected Amazon Pharmacy to...,https://www.fool.com/investing/2024/03/16/eli-...
4,4,MSFT,2024-03-16 06:00:00-04:00,Billionaire Investor Stanley Druckenmiller Has...,Druckenmiller's Duquesne Capital Management fu...,https://www.fool.com/investing/2024/03/16/bill...
...,...,...,...,...,...,...
14807,14807,AMZN,2024-11-09 11:15:00-05:00,Where Will Amazon Be in 5 Years?,It's making incredible progress throughout its...,https://www.fool.com/investing/2024/11/09/wher...
14808,14808,NVDA,2024-11-09 12:00:00-05:00,"With Nvidia Replacing Intel in the Dow Jones, ...",Some Dow Jones components are far from passive...,https://www.fool.com/investing/2024/11/09/nvid...
14809,14809,NVDA,2024-11-09 12:00:00-05:00,"With Nvidia Replacing Intel in the Dow Jones, ...",Some Dow Jones components are far from passive...,https://www.fool.com/investing/2024/11/09/nvid...
14810,14810,MSFT,2024-11-09 18:02:00-05:00,Meet 3 Supercharged Growth Stocks That Could P...,"The much-ballyhooed ""Red Wave"" came to pass. T...",https://www.fool.com/investing/2024/11/09/3-su...


In [30]:
df_news_csv = pd.read_csv('/Users/rishabhbhardwaj/Desktop/Bootcamp project/news data/news_headlines_2000-2022.csv', encoding='ISO-8859-1')

In [37]:
df_news_csv

,Unnamed: 0.1,Unnamed: 0,date,headline
0,0,0,2000-01-03,A 'hindrance to operations': extracts from the...
1,1,1,2000-01-03,Flintoff injury piles on woe for England
2,2,2,2000-01-03,Derby raise a glass to Strupar's debut double
3,3,3,2000-01-03,Kohl's successor drawn into scandal
4,4,4,2000-01-03,Hughes' instant hit buoys Blues
...,...,...,...,...
161849,161849,161849,2022-09-22,Cleaner Was Dead In Belk Bathroom For 4 Days B...
161850,161850,161850,2022-09-22,Woman Who Called Cops On Black Bird-Watcher Lo...
161851,161851,161851,2022-09-23,The Funniest Tweets From Parents This Week (Se...
161852,161852,161852,2022-09-23,"American Airlines Flyer Charged, Banned For Li..."


In [31]:
df_news_recent = pd.read_csv('/Users/rishabhbhardwaj/Desktop/Bootcamp project/bbc_news.csv')

In [13]:
df_news_recent

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,"Mon, 07 Mar 2022 08:01:56 GMT",https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,"Sun, 06 Mar 2022 22:49:58 GMT",https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',"Mon, 07 Mar 2022 00:14:42 GMT",https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,"Mon, 07 Mar 2022 00:05:40 GMT",https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,"Mon, 07 Mar 2022 08:15:53 GMT",https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...
...,...,...,...,...,...
41220,Gauff wins WTA Finals after epic battle with Z...,"Sat, 09 Nov 2024 19:53:13 GMT",https://www.bbc.com/sport/tennis/articles/cj6k...,https://www.bbc.com/sport/tennis/articles/cj6k...,Coco Gauff shows supreme determination to come...
41221,Comedian Godley to go on 'final tour' before f...,"Sat, 09 Nov 2024 17:34:48 GMT",https://www.bbc.com/news/articles/cvgkqpwyr05o#9,https://www.bbc.com/news/articles/cvgkqpwyr05o,The comic's hearse will travel through Edinbur...
41222,Child killed by falling tree branch,"Sat, 09 Nov 2024 20:56:16 GMT",https://www.bbc.com/news/articles/c1lg8jj9dyjo#9,https://www.bbc.com/news/articles/c1lg8jj9dyjo,"Emergency services were called to Banstead, Su..."
41223,What's at stake in the Irish general election?,"Sat, 09 Nov 2024 06:39:32 GMT",https://www.bbc.com/news/articles/c87059nq999o#9,https://www.bbc.com/news/articles/c87059nq999o,"After growing speculation, the Irish prime min..."


In [32]:
df_news_recent['pubDate'] = pd.to_datetime(df_news_recent['pubDate'])

In [21]:
df_news_recent

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,2022-03-07 08:01:56,https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,2022-03-06 22:49:58,https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',2022-03-07 00:14:42,https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,2022-03-07 00:05:40,https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,2022-03-07 08:15:53,https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...
...,...,...,...,...,...
41220,Gauff wins WTA Finals after epic battle with Z...,2024-11-09 19:53:13,https://www.bbc.com/sport/tennis/articles/cj6k...,https://www.bbc.com/sport/tennis/articles/cj6k...,Coco Gauff shows supreme determination to come...
41221,Comedian Godley to go on 'final tour' before f...,2024-11-09 17:34:48,https://www.bbc.com/news/articles/cvgkqpwyr05o#9,https://www.bbc.com/news/articles/cvgkqpwyr05o,The comic's hearse will travel through Edinbur...
41222,Child killed by falling tree branch,2024-11-09 20:56:16,https://www.bbc.com/news/articles/c1lg8jj9dyjo#9,https://www.bbc.com/news/articles/c1lg8jj9dyjo,"Emergency services were called to Banstead, Su..."
41223,What's at stake in the Irish general election?,2024-11-09 06:39:32,https://www.bbc.com/news/articles/c87059nq999o#9,https://www.bbc.com/news/articles/c87059nq999o,"After growing speculation, the Irish prime min..."


In [33]:
# Ensure 'pubDate' column is of string type, then slice the last 14 characters
df_news_recent['pubDate'] = df_news_recent['pubDate'].astype(str).str.slice(stop=-9)

# Display the DataFrame to verify the result
df_news_recent

,title,pubDate,guid,link,description
0,Ukraine: Angry Zelensky vows to punish Russian...,2022-03-07,https://www.bbc.co.uk/news/world-europe-60638042,https://www.bbc.co.uk/news/world-europe-606380...,The Ukrainian president says the country will ...
1,War in Ukraine: Taking cover in a town under a...,2022-03-06,https://www.bbc.co.uk/news/world-europe-60641873,https://www.bbc.co.uk/news/world-europe-606418...,"Jeremy Bowen was on the frontline in Irpin, as..."
2,Ukraine war 'catastrophic for global food',2022-03-07,https://www.bbc.co.uk/news/business-60623941,https://www.bbc.co.uk/news/business-60623941?a...,One of the world's biggest fertiliser firms sa...
3,Manchester Arena bombing: Saffie Roussos's par...,2022-03-07,https://www.bbc.co.uk/news/uk-60579079,https://www.bbc.co.uk/news/uk-60579079?at_medi...,The parents of the Manchester Arena bombing's ...
4,Ukraine conflict: Oil price soars to highest l...,2022-03-07,https://www.bbc.co.uk/news/business-60642786,https://www.bbc.co.uk/news/business-60642786?a...,Consumers are feeling the impact of higher ene...
...,...,...,...,...,...
41220,Gauff wins WTA Finals after epic battle with Z...,2024-11-09,https://www.bbc.com/sport/tennis/articles/cj6k...,https://www.bbc.com/sport/tennis/articles/cj6k...,Coco Gauff shows supreme determination to come...
41221,Comedian Godley to go on 'final tour' before f...,2024-11-09,https://www.bbc.com/news/articles/cvgkqpwyr05o#9,https://www.bbc.com/news/articles/cvgkqpwyr05o,The comic's hearse will travel through Edinbur...
41222,Child killed by falling tree branch,2024-11-09,https://www.bbc.com/news/articles/c1lg8jj9dyjo#9,https://www.bbc.com/news/articles/c1lg8jj9dyjo,"Emergency services were called to Banstead, Su..."
41223,What's at stake in the Irish general election?,2024-11-09,https://www.bbc.com/news/articles/c87059nq999o#9,https://www.bbc.com/news/articles/c87059nq999o,"After growing speculation, the Irish prime min..."


In [34]:
df_news_recent = df_news_recent.drop(columns=['guid','link','description'],axis=1)

In [35]:
df_news_recent = df_news_recent.rename(columns={'title':'headline', 'pubDate': 'date'})

In [38]:
df_news_recent = df_news_recent[df_news_recent['date'] > '2022-09-23']

In [39]:
df_news_recent

,headline,date
8279,Roger Federer: 20-time Grand Slam champion ret...,2022-09-24
8282,Iran protests: US to ease internet curbs for I...,2022-09-24
8296,Jhulan Goswami: The highest wicket-taker in wo...,2022-09-24
8305,Max Whitlock: Olympic champion felt like 'comp...,2022-09-24
8307,Presidents Cup: Team USA extend lead over Inte...,2022-09-24
...,...,...
41220,Gauff wins WTA Finals after epic battle with Z...,2024-11-09
41221,Comedian Godley to go on 'final tour' before f...,2024-11-09
41222,Child killed by falling tree branch,2024-11-09
41223,What's at stake in the Irish general election?,2024-11-09


In [43]:
df_news_combined = pd.concat([df_news_csv, df_news_recent],ignore_index=True)

In [44]:
# Convert 'Date' column to datetime format
df_news_combined['date'] = pd.to_datetime(df_news_combined['date'])

# Remove the time part, keeping only the date
df_news_combined['date'] = df_news_combined['date'].dt.date

In [47]:
df_news_combined.drop(columns=['Unnamed: 0.1','Unnamed: 0'])

,date,headline
0,2000-01-03,A 'hindrance to operations': extracts from the...
1,2000-01-03,Flintoff injury piles on woe for England
2,2000-01-03,Derby raise a glass to Strupar's debut double
3,2000-01-03,Kohl's successor drawn into scandal
4,2000-01-03,Hughes' instant hit buoys Blues
...,...,...
194755,2024-11-09,Gauff wins WTA Finals after epic battle with Z...
194756,2024-11-09,Comedian Godley to go on 'final tour' before f...
194757,2024-11-09,Child killed by falling tree branch
194758,2024-11-09,What's at stake in the Irish general election?


In [293]:
# Define the list of allowed tickers
allowed_tickers = ['AMZN', 'MSFT', 'AAPL', 'NVDA', 'GOOG', 'GOOGL']

# Filter the DataFrame to keep only rows where 'Ticker' is in the allowed list
df_news_combined = df_news_combined[df_news_combined['Ticker'].isin(allowed_tickers)]


,Ticker,Date,Headline,Unnamed: 0


In [294]:
# Replace 'GOOGL' with 'GOOG' in the 'Ticker' column
df_news_combined['Ticker'] = df_news_combined['Ticker'].replace('GOOGL', 'GOOG')

# Display the updated DataFrame
print(df_news_combined)


       Ticker        Date                                           Headline  \
5        AMZN  2011-05-02  Whole Foods (WFMI) -5.2% following a downgrade...   
74       MSFT  2011-05-10  The likely winners in Microsoft's (MSFT -1.4%)...   
77       MSFT  2011-05-10  Microsoft (MSFT -1.2%) and Skype signed their ...   
86       MSFT  2011-05-10                                                NaN   
109      AMZN  2011-05-12  Amazon.com (AMZN -1.7%) shares slip as comment...   
...       ...         ...                                                ...   
192490   AMZN  2024-11-09                   Where Will Amazon Be in 5 Years?   
192491   NVDA  2024-11-09  With Nvidia Replacing Intel in the Dow Jones, ...   
192492   NVDA  2024-11-09  With Nvidia Replacing Intel in the Dow Jones, ...   
192493   MSFT  2024-11-09  Meet 3 Supercharged Growth Stocks That Could P...   
192494   MSFT  2024-11-09  Meet 3 Supercharged Growth Stocks That Could P...   

        Unnamed: 0  
5              NaN

In [48]:
df_news_combined.to_csv('news_headlines-2000-2024.csv')

In [216]:
df_news_combined = pd.read_csv('/Users/rishabhbhardwaj/Desktop/Bootcamp project/news data/news_headlines_2000-2022.csv')
df_news_combined['date'] = df_news_combined['date'].str.replace(" ", "")

df_news_combined.to_csv('news_headlines_2000-2022.csv')